In [ ]:
import pandas as pd

df = pd.read_csv('../dataset_potiguana_final.csv')
df.drop('reference', axis=1, inplace=True)
df.head(2)

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """
Você é um servidor da UFRN responsável por instruir alunos sobre questões acadêmicas do regulamento dos cursos de graduação.
Você deve responder a resposta correta baseada na questão e contexto abaixo. Por favor, siga as instruções:

1. Pergunta: {pergunta}

2. Contexto: {contexto}

3. Instruções:
    - Analise cuidadosamente a questão e o contexto fornecido.
    - Formule uma resposta abrangente e precisa baseada apenas nas informações fornecidas no contexto.
    - Certifique-se de que sua resposta aborda diretamente a pergunta.
    - Inclua todas as informações relevantes do contexto, mas não adicione nenhum conhecimento externo.
    - Se o contexto não contiver informações suficientes para responder completamente à pergunta, declare isso claramente e forneça a melhor resposta parcial possível.
    - Use um tom formal e objetivo.

"""

prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
questions = []
for _, row in df.iterrows():
    questions.append(row.to_dict())


In [ ]:
from time import sleep


response_list = []

for data in questions:
    try:
        if not 'reference' in data.keys():
            prompt = prompt_template.format(pergunta=data['user_input'], contexto=data['reference_contexts'])
            response = llm.invoke(prompt)
            print(response)
            data['reference'] = response
            response_list.append(data)
    except Exception as exc:
        print(exc)
        sleep(2)

questions = response_list

In [ ]:
df_final = pd.DataFrame(questions)
df_final.head(2)

In [ ]:
df_final = df_final.reindex(['user_input', 'reference_contexts','reference', 'synthesizer_name'], axis=1)
df_final.head(2)

In [ ]:
df_final.to_csv('../dataset_potiguana.csv', index=False)